<a href="https://colab.research.google.com/github/wcl20/Tensorflow-Facial-Expression/blob/master/FacialExpression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup Environment

In [0]:
!pip install tensorflow-gpu==2.0.0

# 2. Import Project Dependencies

In [58]:
import numpy as np
import pandas as pd
import tensorflow as tf

import zipfile

tf.__version__

'2.0.0'

# 3. Preprocess Data

## 3.1 Load Data

In [0]:
# Extract Files
zip_object = zipfile.ZipFile(file="./facialExpressions.zip", mode="r")
zip_object.extractall("./data")
zip_object.close()

## 3.2 Load Training Data

In [50]:
df = pd.read_csv("data/icml_face_data.csv")
df.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [51]:
# View Training Data Distribution
training_data = df[df[" Usage"] == "Training"].drop(" Usage", axis=1)
np.unique(training_data["emotion"], return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([3995,  436, 4097, 7215, 4830, 3171, 4965]))

In [52]:
# Duplicate class 1 ten times
class1 = training_data[training_data["emotion"] == 1]
class1_duplicate = pd.concat([class1] * 10, ignore_index=True)
training_data = pd.concat([training_data, class1_duplicate], ignore_index=True)

np.unique(training_data["emotion"], return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([3995, 4796, 4097, 7215, 4830, 3171, 4965]))

In [53]:
# Convert pixel strings to numpy arrays
pixels = []
for string in training_data.iloc[:,1]:
  pixels.append(np.array(string.split(), dtype=np.float32))
training_data.drop(" pixels", axis=1, inplace=True)
training_data = pd.concat([training_data, pd.DataFrame(pixels)], axis=1)

training_data.head()

,emotion,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2264,2265,2266,2267,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282,2283,2284,2285,2286,2287,2288,2289,2290,2291,2292,2293,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,0,70.0,80.0,82.0,72.0,58.0,58.0,60.0,63.0,54.0,58.0,60.0,48.0,89.0,115.0,121.0,119.0,115.0,110.0,98.0,91.0,84.0,84.0,90.0,99.0,110.0,126.0,143.0,153.0,158.0,171.0,169.0,172.0,169.0,165.0,129.0,110.0,113.0,107.0,95.0,...,20.0,31.0,40.0,46.0,65.0,88.0,108.0,110.0,125.0,149.0,157.0,153.0,162.0,164.0,158.0,159.0,154.0,140.0,78.0,21.0,11.0,61.0,144.0,168.0,173.0,157.0,138.0,150.0,148.0,132.0,159.0,182.0,183.0,136.0,106.0,116.0,95.0,106.0,109.0,82.0
1,0,151.0,150.0,147.0,155.0,148.0,133.0,111.0,140.0,170.0,174.0,182.0,154.0,153.0,164.0,173.0,178.0,185.0,185.0,189.0,187.0,186.0,193.0,194.0,185.0,183.0,186.0,180.0,173.0,166.0,161.0,147.0,133.0,172.0,151.0,114.0,161.0,161.0,146.0,131.0,...,150.0,129.0,143.0,135.0,115.0,102.0,111.0,124.0,112.0,109.0,132.0,146.0,135.0,149.0,148.0,143.0,163.0,156.0,159.0,150.0,139.0,128.0,116.0,125.0,133.0,109.0,130.0,147.0,130.0,121.0,105.0,108.0,95.0,108.0,102.0,67.0,171.0,193.0,183.0,184.0
2,2,231.0,212.0,156.0,164.0,174.0,138.0,161.0,173.0,182.0,200.0,106.0,38.0,39.0,74.0,138.0,161.0,164.0,179.0,190.0,201.0,210.0,216.0,220.0,224.0,222.0,218.0,216.0,213.0,217.0,220.0,220.0,218.0,217.0,212.0,174.0,160.0,162.0,160.0,139.0,...,225.0,220.0,215.0,207.0,199.0,167.0,108.0,151.0,122.0,88.0,71.0,84.0,120.0,127.0,105.0,76.0,71.0,78.0,90.0,106.0,123.0,146.0,155.0,148.0,130.0,141.0,119.0,69.0,54.0,89.0,104.0,138.0,152.0,122.0,114.0,101.0,97.0,88.0,110.0,152.0
3,4,24.0,32.0,36.0,30.0,32.0,23.0,19.0,20.0,30.0,41.0,21.0,22.0,32.0,34.0,21.0,19.0,43.0,52.0,13.0,26.0,40.0,59.0,65.0,12.0,20.0,63.0,99.0,98.0,98.0,111.0,75.0,62.0,41.0,73.0,118.0,140.0,192.0,186.0,187.0,...,124.0,150.0,161.0,167.0,169.0,167.0,162.0,164.0,169.0,181.0,172.0,168.0,169.0,170.0,174.0,179.0,180.0,182.0,196.0,208.0,226.0,230.0,140.0,113.0,114.0,125.0,135.0,141.0,172.0,214.0,174.0,126.0,132.0,132.0,133.0,136.0,139.0,142.0,143.0,142.0
4,6,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,15.0,23.0,28.0,48.0,50.0,58.0,84.0,115.0,127.0,137.0,142.0,151.0,156.0,155.0,149.0,153.0,152.0,157.0,160.0,162.0,159.0,145.0,121.0,83.0,58.0,48.0,...,18.0,27.0,19.0,13.0,11.0,22.0,37.0,48.0,55.0,67.0,88.0,95.0,112.0,118.0,123.0,141.0,157.0,174.0,181.0,185.0,184.0,179.0,176.0,171.0,155.0,128.0,105.0,10.0,0.0,0.0,12.0,34.0,31.0,31.0,31.0,27.0,31.0,30.0,29.0,30.0


In [54]:
x_train = training_data.iloc[:,1:].values.reshape(-1, 48, 48, 1)
y_train = training_data.iloc[:,0].values

x_train.shape

(33069, 48, 48, 1)

## 3.3 Load Testing Data

In [59]:
# Load Testing Data 
testing_data = df[df[" Usage"] == "PrivateTest"].drop(" Usage", axis=1).reset_index(drop=True)
testing_data.head()

,emotion,pixels
0,0,170 118 101 88 88 75 78 82 66 74 68 59 63 64 6...
1,5,7 5 8 6 7 3 2 6 5 4 4 5 7 5 5 5 6 7 7 7 10 10 ...
2,6,232 240 241 239 237 235 246 117 24 24 22 13 12...
3,4,200 197 149 139 156 89 111 58 62 95 113 117 11...
4,2,40 28 33 56 45 33 31 78 152 194 200 186 196 20...


In [60]:
# Convert pixel strings to numpy arrays
pixels = []
for string in testing_data.iloc[:,1]:
  pixels.append(np.array(string.split(), dtype=np.float32))
testing_data.drop(" pixels", axis=1, inplace=True)
testing_data = pd.concat([testing_data, pd.DataFrame(pixels)], axis=1)

testing_data.head()

,emotion,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2264,2265,2266,2267,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282,2283,2284,2285,2286,2287,2288,2289,2290,2291,2292,2293,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,0,170.0,118.0,101.0,88.0,88.0,75.0,78.0,82.0,66.0,74.0,68.0,59.0,63.0,64.0,65.0,90.0,89.0,73.0,80.0,80.0,85.0,88.0,95.0,117.0,132.0,146.0,139.0,152.0,164.0,181.0,182.0,194.0,195.0,162.0,142.0,147.0,151.0,148.0,158.0,...,66.0,68.0,98.0,88.0,76.0,95.0,114.0,102.0,109.0,115.0,121.0,130.0,139.0,142.0,145.0,147.0,149.0,150.0,157.0,168.0,168.0,168.0,167.0,160.0,144.0,128.0,126.0,120.0,112.0,143.0,210.0,210.0,222.0,222.0,204.0,190.0,174.0,159.0,133.0,131.0
1,5,7.0,5.0,8.0,6.0,7.0,3.0,2.0,6.0,5.0,4.0,4.0,5.0,7.0,5.0,5.0,5.0,6.0,7.0,7.0,7.0,10.0,10.0,8.0,6.0,8.0,11.0,12.0,15.0,9.0,7.0,16.0,21.0,9.0,8.0,13.0,18.0,23.0,20.0,23.0,...,11.0,28.0,19.0,12.0,12.0,11.0,9.0,9.0,11.0,10.0,10.0,14.0,18.0,19.0,18.0,14.0,18.0,24.0,21.0,13.0,20.0,58.0,82.0,90.0,93.0,90.0,87.0,79.0,85.0,75.0,41.0,37.0,76.0,88.0,84.0,88.0,82.0,72.0,57.0,52.0
2,6,232.0,240.0,241.0,239.0,237.0,235.0,246.0,117.0,24.0,24.0,22.0,13.0,12.0,14.0,9.0,8.0,5.0,6.0,8.0,9.0,9.0,7.0,7.0,6.0,4.0,4.0,4.0,5.0,7.0,4.0,10.0,12.0,12.0,15.0,20.0,14.0,17.0,36.0,206.0,...,174.0,152.0,142.0,100.0,28.0,29.0,27.0,28.0,25.0,78.0,20.0,0.0,18.0,24.0,11.0,19.0,36.0,35.0,37.0,30.0,56.0,200.0,208.0,162.0,38.0,101.0,91.0,75.0,107.0,139.0,64.0,21.0,89.0,68.0,46.0,67.0,2.0,4.0,4.0,9.0
3,4,200.0,197.0,149.0,139.0,156.0,89.0,111.0,58.0,62.0,95.0,113.0,117.0,116.0,116.0,112.0,111.0,96.0,86.0,99.0,113.0,120.0,117.0,116.0,109.0,114.0,125.0,132.0,136.0,136.0,132.0,128.0,128.0,134.0,132.0,132.0,136.0,138.0,137.0,134.0,...,79.0,89.0,99.0,105.0,106.0,115.0,117.0,122.0,134.0,139.0,137.0,143.0,148.0,149.0,149.0,151.0,147.0,149.0,157.0,158.0,146.0,145.0,147.0,137.0,136.0,133.0,128.0,128.0,122.0,128.0,142.0,161.0,185.0,185.0,107.0,146.0,144.0,149.0,33.0,0.0
4,2,40.0,28.0,33.0,56.0,45.0,33.0,31.0,78.0,152.0,194.0,200.0,186.0,196.0,207.0,194.0,185.0,197.0,185.0,176.0,167.0,166.0,162.0,160.0,151.0,145.0,142.0,135.0,134.0,134.0,131.0,125.0,122.0,101.0,72.0,48.0,32.0,27.0,22.0,22.0,...,139.0,133.0,134.0,140.0,125.0,126.0,144.0,108.0,94.0,80.0,60.0,51.0,52.0,34.0,43.0,47.0,40.0,37.0,33.0,22.0,24.0,26.0,26.0,27.0,26.0,28.0,26.0,29.0,31.0,34.0,25.0,26.0,29.0,26.0,28.0,25.0,24.0,24.0,29.0,26.0


In [61]:
x_test = testing_data.iloc[:,1:].values.reshape(-1, 48, 48, 1)
y_test = testing_data.iloc[:,0].values

x_test.shape

(3589, 48, 48, 1)

## 3.4 Image Normalization

In [0]:
x_train = x_train / 255.
x_test = x_test / 255.

# 4. Convolution Neural Network 

## 4.1 Building the Model

In [63]:
# Building CNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[48, 48, 1]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)         

In [0]:
# Compile Model
model.compile(loss="sparse_categorical_crossentropy", optimizer="Adam", metrics=["sparse_categorical_accuracy"])

## 4.2 Training the Model

In [65]:
model.fit(x_train, y_train, epochs=5, batch_size=64)

Train on 33069 samples
Epoch 1/5
33069/33069 [==============================] - 165s 5ms/sample - loss: 1.7116 - sparse_categorical_accuracy: 0.3339
Epoch 2/5
33069/33069 [==============================] - 164s 5ms/sample - loss: 1.2714 - sparse_categorical_accuracy: 0.5234
Epoch 3/5
33069/33069 [==============================] - 164s 5ms/sample - loss: 1.0486 - sparse_categorical_accuracy: 0.6061
Epoch 4/5
33069/33069 [==============================] - 164s 5ms/sample - loss: 0.9318 - sparse_categorical_accuracy: 0.6476
Epoch 5/5
33069/33069 [==============================] - 164s 5ms/sample - loss: 0.8477 - sparse_categorical_accuracy: 0.6822


## 4.3 Evaluation the Model

In [68]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy}")

3589/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

## 4.4 Saving the Model

In [0]:
# Save model
json = model.to_json()
with open("model.json", "w") as f:
    f.write(json)

In [0]:
# Save weights
model.save_weights("model.h5")